In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime

import PIL
# Rmb to run "pip install pillow"
# from ResNet import Bottleneck, ResNet, ResNet50

In [ ]:
import os
from os import listdir

In [ ]:
# Folders
train_FAKE_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/GenImage/imagenet_ai_0419_biggan/train/FAKE"
train_REAL_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/GenImage/imagenet_ai_0419_biggan/train/REAL"

test_FAKE_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/GenImage/imagenet_ai_0419_biggan/val/ai"
test_REAL_folder_path = "C:/Users/tan_l/Workspace/CS5242/Group Project/Data/GenImage/imagenet_ai_0419_biggan/val/nature"

## Prepare TRAIN and TEST Tensor dataset (Let label for Real be 1 and label for Fake be 0)

In [ ]:
def ignore_alpha(image):
    if image.mode == 'RGBA':
        r, g, b, _ = image.split()
        return PIL.Image.merge("RGB", (r, g, b))
    elif image.mode == 'RGB':
        return image
    else:
        return image.convert("RGB")

transform_train_V1 = transforms.Compose([
    ignore_alpha,
    transforms.Resize((224,224)),
    transforms.CenterCrop((224,224)),
    transforms.RandomHorizontalFlip(), # To introduce image orientation variation
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

])

transform_test_V1 = transforms.Compose([
    ignore_alpha,
    transforms.Resize((224, 224)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])



In [ ]:
train_tensor_list =[]
#Format = list of tuples with each tuple in this format ((tensor version of image), label)

# FAKE portion
for images in os.listdir(train_FAKE_folder_path):
    # print(images)
    # if images.endswith(".png"):
    img = PIL.Image.open(f"{train_FAKE_folder_path}/%s" % images)
    # train_tensor_list.append(transform_train_V1(img))
    train_tensor_list.append((transform_train_V1(img), 0))
# REAL portion
for images in os.listdir(train_REAL_folder_path):
    # if images.endswith(".png"):
    img = PIL.Image.open(f"{train_REAL_folder_path}/%s" % images)
    # train_tensor_list.append(transform_train_V1(img))
    train_tensor_list.append((transform_train_V1(img), 1))

In [ ]:
test_tensor_list =[]
#Format = list of tuples with each tuple in this format ((tensor version of image), label)

# FAKE portion
for images in os.listdir(test_FAKE_folder_path):
    # print(images)
    img = PIL.Image.open(f"{test_FAKE_folder_path}/%s" % images)
    # test_tensor_list.append(transform_test_V1(img))
    test_tensor_list.append((transform_test_V1(img),0))
# REAL portion

for images in os.listdir(test_REAL_folder_path):
    # print(images)
    img = PIL.Image.open(f"{test_REAL_folder_path}/%s" % images)
    # test_tensor_list.append(transform_test_V1(img))
    test_tensor_list.append((transform_test_V1(img),1))

In [ ]:
print(len(train_tensor_list))
print(len(test_tensor_list))

33000
12000


In [ ]:
## LOAD DATA to trainloader and testloader

# train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train_tensor_list, batch_size=128, shuffle=True, num_workers=2)

# test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test_tensor_list, batch_size=128,shuffle=False, num_workers=2)

for X, y in trainloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([128, 3, 224, 224])
Shape of y: torch.Size([128]) torch.int64


### DenseNET Model

In [ ]:
import torch

# input_batch = torch.stack(train_tensor_list)
# print(input_batch)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet161', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

Using cuda device


Using cache found in C:\Users\tan_l/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\tan_l\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\tan_l\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 10
print("Started:", datetime.now())
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Started: 2024-04-09 14:47:35.901691
Epoch 1
-------------------------------
loss: 15.491557  [  128/33000]
loss: 0.269792  [12928/33000]
loss: 0.237850  [25728/33000]
Test Error: 
 Accuracy: 98.1%, Avg loss: 0.083651 

Epoch 2
-------------------------------
loss: 0.105236  [  128/33000]
loss: 0.061983  [12928/33000]
loss: 0.059349  [25728/33000]
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.049591 

Epoch 3
-------------------------------
loss: 0.081822  [  128/33000]
loss: 0.071767  [12928/33000]
loss: 0.036749  [25728/33000]
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.036672 

Epoch 4
-------------------------------
loss: 0.054950  [  128/33000]
loss: 0.042358  [12928/33000]
loss: 0.022020  [25728/33000]
Test Error: 
 Accuracy: 99.3%, Avg loss: 0.032460 

Epoch 5
-------------------------------
loss: 0.021797  [  128/33000]
loss: 0.048122  [12928/33000]
loss: 0.020998  [25728/33000]
Test Error: 
 Accuracy: 99.5%, Avg loss: 0.023592 

Epoch 6
-------------------------------
loss: 0.02279

In [ ]:
# Save model

print("Finished:", datetime.now())
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Finished: 2024-04-10 06:44:35.280504
Saved PyTorch Model State to model.pth
